# Unified LLM Generation Pipeline

This notebook provides an interface to the unified generation pipeline.
All logic lives in `generation.py` - this notebook is for interactive testing and batch runs.

The pipeline handles both Paleolithic and Holocene people automatically based on the sampled birth year.

In [2]:
import dill
from tqdm import tqdm

from generation import (
    generate_person,
    generate_batch,
    generate_batch_parallel,
    # Individual steps if needed
    generate_geography,
    generate_demographics,
    generate_structured_incidents,
    generate_historical_context,
    generate_name,
    generate_narrative_plan,
    generate_narrative,
    run_pipeline,
    reset_to_stage
)
from llm_utils import GenerationContext, extract_json

from person import sample_year, sample_person, Person

import copy

# Batch Generation

In [4]:
'''test_people = generate_batch_parallel(n=150, model="gpt-5.2", workers=50)

Sampling 150 people...
Generating 150 people with 50 parallel workers...


100%|█████████████████████████████████████████| 150/150 [13:15<00:00,  5.30s/it]

Done: 150 generated


In [7]:
'''with open('batch2_0100_0249.pkl', 'wb') as f:
    dill.dump(test_people,f)
%run export.py batch2_0100_0249.pkl --start-index 100

Loading people from batch2_0100_0249.pkl...
Found 150 people
Removing existing markdown files from ../_lives_pending...
  Removed 7 files
  Exported 10 people...
  Exported 20 people...
  Exported 30 people...
  Exported 40 people...
  Exported 50 people...
  Exported 60 people...
  Exported 70 people...
  Exported 80 people...
  Exported 90 people...
  Exported 100 people...
  Exported 110 people...
  Exported 120 people...
  Exported 130 people...
  Exported 140 people...
  Exported 150 people...

Successfully exported 150 people to ../_lives_pending
  Index range: 0100 - 0249


# Checking existing generation

In [20]:
with open('batch1_0000_0099.pkl', 'rb') as f:
    people = dill.load(f)

with open('batch2_0100_0249.pkl', 'rb') as f:
    people += dill.load(f)

In [25]:
ages = []
for person in people:
    if person.lifestyle == 'Rural' and person.birth_year<1500:
        ages += [person.age_at_death]

In [26]:
import numpy as np

In [27]:
np.average(ages)

24.610294117647058

In [28]:
len(ages)

136